# Object Detection Module Using Pre-Trained Networks for Security Systems
This module demonstrates how to implement an object detection system using Ultralytics' YOLOv5, a pre-trained network model for security purposes. The system will detect potentially hazardous objects, such as knives, guns, or fire, and can be integrated into a larger security framework to enhance monitoring and threat detection.

## Import required packages

In [ ]:
import torch 
import matplotlib.pyplot as plt
from ultralytics import YOLO
import cv2
import os
import yaml

## Load the Pre-Trained YOLOv5 Model
**YOLOv5s** (small version) will be used for object detection. Other versions like `yolov5m.pt` for medium, or `yolov5x.pt` for extra large can be choosen instead.

In [ ]:
# load the YOLOv5 model
model = YOLO('yolov8n.pt')

## Create YAML File for Fine-Tuning

In [ ]:
def create_yaml():
    # create yaml file to fine tune model 
    data = {
        # 'path': f'{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/fine-tune-dataset',
        'train': f'{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/codes/fine-tune-dataset/datasets/train',
        'val': f'{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/codes/fine-tune-dataset/datasets/val',
        'nc': 2,
        'names': ['knife', 'pistol'],
    }
    
    
    with open('fine-tune.yaml', 'w+') as f:
    
        yaml.dump(data, f)

## Fine-Tune the Model
Function to fine tune the YOLOv8n model with knife and gun images. Tuned model will be save to save time since it is computationally intensive.

**NOTE** </br>
The `device=0` parameter in **line 3** below is used only if pytorch is setup with cuda. If it is not setup to use cuda do remove.

In [ ]:
def fine_tune_model():
    create_yaml()
    model.train(data=f'{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/codes/fine-tune.yaml', epochs=50, imgsz=640, batch=16, device=0)
    model.save('yolo_fine_tuned.pt')
    print("Model fine-tunning complete. Model saved as 'yolo_fine_tuned.pt'.")

## Load the Fine-Tuned Model

In [ ]:
def load_model():
    model = YOLO('yolo_fine_tuned.pt')
    return model

## Evaluate the Model 

In [ ]:
def evaluate_model(model):
    results = model.val(data=f'{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/codes/fine-tune.yaml')
    print(results.box.maps)

## Perform Object Detection on an image
This section will load an image and run the object detection model. the model will automatically annotate the image with bounding boxes and class labels for each detected object.

In [ ]:
def detect_hazardous_obj(model, img_path, output_dir='./detected_imgs'):
    # perform object detection
    results = model(img_path)

    # retrieve the annotated image (with bounding boxes and labels)
    annotated_img = results[0].plot()

    # display the image
    plt.imshow(annotated_img)
    plt.axis('off')
    plt.show()

    # Save the annotated image
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, 'annotated_img.jpg')
    cv2.imwrite(output_path, annotated_img)
    print(f"Annotated image saved as '{output_path}'.")
    

## Example Usage

In [ ]:
# Fine-tune the model (uncomment this to fine tune the yolo model)
# fine_tune_model()

# load the fine-tuned model
model = load_model()

create_yaml()
# evaluate the model 
evaluate_model(model)

# path to the image to be tested
img_path = './test_imgs/guy_w_gun.jpg'

# detect hazardous objects in the test image
detect_hazardous_obj(model, img_path)